In [1]:
!pip install pandas -q

import pandas as pd

# File paths
train_file = "./train_low_std_range_very_low_std_standardized_log1p.csv"
test_file = "./test_low_std_range_very_low_std_standardized_log1p.csv"

# Load the datasets
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

# Display the first few rows of each dataset
print("Train Data:")
print(train_df.info())

print("\nTest Data:")
print(test_df.info())



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Train Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Series1  800000 non-null  float64
 1   Series2  800000 non-null  float64
 2   Series3  800000 non-null  float64
 3   Label    800000 non-null  float64
dtypes: float64(4)
memory usage: 24.4 MB
None

Test Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Series1  200000 non-null  float64
 1   Series2  200000 non-null  float64
 2   Series3  200000 non-null  float64
 3   Label    200000 non-null  float64
dtypes: float64(4)
memory usage: 6.1 MB
None


In [2]:
train_df.describe()

,Series1,Series2,Series3,Label
count,800000.000000,800000.000000,800000.000000,8.000000e+05
mean,-0.000832,0.000195,0.000584,3.674458e-01
std,0.999939,0.999988,0.999805,6.178303e-01
min,-2.338787,-1.714816,-1.714816,0.000000e+00
25%,-0.429087,-0.848747,-0.848747,4.289651e-09
50%,0.000225,-0.017321,0.017321,1.990234e-01
75%,0.425705,0.883390,0.883390,4.718445e-01
max,2.338214,1.714816,1.714816,5.991320e+00


In [3]:
# install norse and pytorch
!pip install sinabs


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch

class DataFrameDataset(Dataset):
    def __init__(self, dataframe):
        self.inputval = dataframe["Series1"]
        self.mean = dataframe["Series2"]
        self.std = dataframe["Series3"]
        self.labels = dataframe["Label"]

    def __len__(self):
        return len(self.inputval)

    def __getitem__(self, idx):
        inputval = self.inputval.iloc[idx]
        mean = self.mean.iloc[idx]
        std = self.std.iloc[idx]
        labels = self.labels.iloc[idx]
        return inputval, mean, std, labels
    
# Create the datasets
train_dataset = DataFrameDataset(train_df)
test_dataset = DataFrameDataset(test_df)


In [5]:
#import numpy

#num_bins = 4/0.01

#min_val = float(train_dataset.labels.min())
#max_val = float(train_dataset.labels.max())

#bins_train = torch.linspace(min_val, max_val, int(num_bins + 1))
#bin_indices_train = numpy.digitize(train_dataset.labels, bins_train, right=True)
##print(bin_indices_train)
#bin_counts = torch.bincount(torch.tensor(bin_indices_train))

# plot bin_counts
#import matplotlib.pyplot as plt
#plt.hist(bin_counts)
#plt.show()
#bin_counts

In [6]:
# We calculate the weights for each bin

#bin_weights = 1.0 / (bin_counts + 1e-6)  # Inverse of bin counts
#plt.hist(bin_weights)
#plt.show()
#print(len(bin_weights))
#print("Summed: ", sum(bin_weights))
#sample_weights = bin_weights[bin_indices]  # Assign weights to each sample

In [7]:
#len(sample_weights)

In [8]:
#from torch.utils.data import WeightedRandomSampler

#sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)


In [9]:
# Encode input data
train_loader = DataLoader(train_dataset, batch_size=64) #, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=64) #, shuffle=True)


In [10]:
import torch.nn as nn
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [11]:
import torch

TRAINING_PREFIX = "long_train_low_range_attempt_with_low_std_feature_scale_and_log1p"

ann = nn.Sequential(
    nn.Linear(3, 256),  # Input layer: 3 features (mu, sigma, x)
    nn.ReLU(),
    #nn.Dropout(0.2),
    nn.Linear(256, 256),  # first hidden layer
    nn.ReLU(),
    #nn.Dropout(0.2),
    nn.Linear(256, 256),  # second hidden layer
    nn.ReLU(),
    #nn.Dropout(0.2),
    nn.Linear(256, 1)    # Output layer: single value for f(x; mu, sigma)
)

ann.load_state_dict(torch.load("results/long_train_low_range_attempt_with_low_std_feature_scale_and_log1p/ann_epoch_1000.pth"))

ann.to(device)

total_params = sum(p.numel() for p in ann.parameters() if p.requires_grad)
print(f"Total Parameters: {total_params}")


Total Parameters: 132865


C:\Users\Lutz\AppData\Local\Temp\ipykernel_56032\114933261.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ann.load_state_dict(torch.load("results/long_train_low_range_

In [12]:
from sinabs.from_torch import from_model

num_time_steps_per_sample = 100

sinabs_model = from_model(ann, input_shape=(3,), add_spiking_output=True, synops=False, num_timesteps=num_time_steps_per_sample)
sinabs_model

Network(
  (spiking_model): Sequential(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (6): Linear(in_features=256, out_features=1, bias=True)
    (spike_output): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
  )
  (analog_model): Sequential(
    (0): Linear(in_features=3, out_features

In [14]:
!pip install torcheval -q

from IPython.display import clear_output
from matplotlib import pyplot as plt
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display
from torcheval.metrics import R2Score


def create_loss_plot():
    fig = go.FigureWidget()
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Train Loss', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Eval Loss', line=dict(color='orange')))

    # Configure layout
    fig.update_layout(title='Training and Evaluation Losses',
                    xaxis_title='Epoch',
                    yaxis_title='Loss',
                    template='plotly_dark')

    # Display the figure widget
    display(fig)
    return fig

def update_loss_plot(fig, train_loss, eval_loss, from_epoch=0):
    if from_epoch != 0:
        with fig.batch_update():
            fig.data[0].x = list(range(from_epoch, len(train_loss)))
            fig.data[0].y = train_loss[from_epoch:]
            fig.data[1].x = list(range(from_epoch, len(eval_loss)))
            fig.data[1].y = eval_loss[from_epoch:]
    elif len(train_loss) < 30:
        with fig.batch_update():
            fig.data[0].x = list(range(len(train_loss)))
            fig.data[0].y = train_loss
            fig.data[1].x = list(range(len(eval_loss)))
            fig.data[1].y = eval_loss
    else:
        with fig.batch_update():
            fig.data[0].x = list(range(len(train_loss) - 30, len(train_loss)))
            fig.data[0].y = train_loss[-30:]
            fig.data[1].x = list(range(len(eval_loss) - 30, len(eval_loss)))
            fig.data[1].y = eval_loss[-30:]        
    

import numpy as np
def gaussian_probability(x, y, z):
    return (1 / (np.sqrt(2 * np.pi) * z)) * np.exp(-((x - y) ** 2) / (2 * z ** 2))

from tqdm.auto import tqdm
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

def do_epoch(loader, model, optimizer, device, n_epochs: int, current_epoch: int, train: bool = False,):
    if train:
        model.train()
    else:
        model.eval()
    epoch_loss = 0
    metric = R2Score(device=device)
    for inputvals, means, stds, labels in tqdm(loader, desc=f"{'Epoch' if train else 'Eval Epoch'} {current_epoch+1}/{n_epochs}"):
        inputs = torch.stack((inputvals, means, stds), dim=1).float()
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        metric.update(outputs.squeeze(), labels)
        if train:
            optimizer.zero_grad()
            loss = F.mse_loss(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
        else:
            loss = F.mse_loss(outputs.squeeze(), labels.float())
        epoch_loss += loss.item()
    return epoch_loss, metric.compute()


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
ann.train()

N_EPOCHS = 1500
LR = 0.05
SAVE_EVERY = 50

# Create subfolder for this loop
import os
import shutil

#if os.path.exists(f"./results/{TRAINING_PREFIX}"):
#    shutil.rmtree(f"./results/{TRAINING_PREFIX}")
#os.makedirs(f"./results/{TRAINING_PREFIX}")

#optim = torch.optim.Adam(ann.parameters(), lr=LR)
optim = torch.optim.SGD(ann.parameters(), lr=LR)

last_loss = 0
last_eval_loss = 0

epoch_loss_list = []
eval_loss_list = [] 

fig = create_loss_plot()

for epoch in range(1001, N_EPOCHS):
    if (epoch + 1) % 30 == 0:
        clear_output(wait=True)
        fig = create_loss_plot()
        update_loss_plot(fig, epoch_loss_list, eval_loss_list)
    
    epoch_loss, train_R2_score = do_epoch(train_loader, ann, optim, device, n_epochs=N_EPOCHS, current_epoch=epoch, train=True)
    eval_epoch_loss, eval_R2_score = do_epoch(test_loader, ann, None, device, n_epochs=N_EPOCHS, current_epoch=epoch, train=False)
    eval_loss_list.append(eval_epoch_loss/len(test_loader))
    print(f"Epoch {epoch+1}, Train Loss: {epoch_loss/len(train_loader)}, Diff: {epoch_loss/len(train_loader) - last_loss}, Eval Loss: {eval_epoch_loss/len(test_loader)}, Diff Eval: {eval_epoch_loss/len(test_loader) - last_eval_loss}, Train R2 Score: {train_R2_score}, Eval R2 Score: {eval_R2_score}")
    last_loss = epoch_loss/len(train_loader)
    last_eval_loss = eval_epoch_loss/len(test_loader)
    epoch_loss_list.append(epoch_loss/len(train_loader))
    update_loss_plot(fig, epoch_loss_list, eval_loss_list)
    if epoch % SAVE_EVERY == 0:
        torch.save(ann.state_dict(), f"./results/{TRAINING_PREFIX}/ann_epoch_{epoch}.pth")
    elif epoch == N_EPOCHS - 1:
        torch.save(ann.state_dict(), f"./results/{TRAINING_PREFIX}/ann_epoch_{epoch}.pth") 

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines',
              'name': 'Train Loss',
              'type': 'scatter',
              'uid': '3e4dd809-b7b7-4e67-bd16-bc14b57071bb',
              'x': [],
              'y': []},
             {'line': {'color': 'orange'},
              'mode': 'lines',
              'name': 'Eval Loss',
              'type': 'scatter',
              'uid': 'e8fb4db2-8ce4-45bf-ad29-605f0aedb773',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Training and Evaluation Losses'},
               'xaxis': {'title': {'text': 'Epoch'}},
               'yaxis': {'title': {'text': 'Loss'}}}
})

Epoch 1500/1500:   0%|          | 0/12500 [00:00<?, ?it/s]

Eval Epoch 1500/1500:   0%|          | 0/3125 [00:00<?, ?it/s]

Epoch 1500, Train Loss: 0.0031474453799414186, Diff: 1.084425184635614e-05, Eval Loss: 0.0029398525845605763, Diff Eval: -0.00019333799878397265, Train R2 Score: 0.9917544722557068, Eval R2 Score: 0.9923216700553894


In [18]:
#torch.save(ann.state_dict(), "largerer_ann_new_data5.pth")


In [19]:
fig = create_loss_plot()
update_loss_plot(fig, epoch_loss_list, eval_loss_list, 30)

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines',
              'name': 'Train Loss',
              'type': 'scatter',
              'uid': 'fdfbc7be-449e-492a-9a0a-52bb34b4f773',
              'x': [],
              'y': []},
             {'line': {'color': 'orange'},
              'mode': 'lines',
              'name': 'Eval Loss',
              'type': 'scatter',
              'uid': '5afdd6f0-2a3e-42b2-9751-5eb3aebc228d',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Training and Evaluation Losses'},
               'xaxis': {'title': {'text': 'Epoch'}},
               'yaxis': {'title': {'text': 'Loss'}}}
})

In [20]:
# Import pickled feature scaler
import pickle

with open("scaler_low_std_range_very_low_std_feature.pkl", "rb") as f:
    feature_scaler = pickle.load(f)


In [21]:
feature_scaler.transform([[0.5, 0.5, 0.5]])

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



array([[ 0.19458858,  0.85740802, -0.00171653]])

In [22]:
transformed = feature_scaler.transform([[0.5, 0.5, 0.5]])
transformed

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



array([[ 0.19458858,  0.85740802, -0.00171653]])

In [23]:
ann.eval()
transformed = feature_scaler.transform([[0.5, 0.5, 0.5]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())


c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(0.7967774953315954)

In [24]:
gaussian_probability(0.5, 0.5, 0.5)

np.float64(0.7978845608028654)

In [25]:
transformed = feature_scaler.transform([[0.1, 0.1, 0.1]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(4.082113893864278)

In [26]:
gaussian_probability(0.1, 0.1, 0.1)

np.float64(3.989422804014327)

In [27]:
transformed = feature_scaler.transform([[0.01, 0.01, 0.01]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(34.66522316888408)

In [28]:
gaussian_probability(0.01, 0.01, 0.01)

np.float64(39.894228040143275)

In [29]:
transformed = feature_scaler.transform([[0.001, 0.001, 0.001]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(237.69300819388917)

In [30]:
gaussian_probability(0.001, 0.001, 0.001)

np.float64(398.94228040143275)

In [31]:
transformed = feature_scaler.transform([[0.9, 0.9, 0.9]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(0.44480746352347655)

In [32]:
gaussian_probability(0.9, 0.9, 0.9)

np.float64(0.44326920044603635)

In [33]:
transformed = feature_scaler.transform([[0.9, 0.7, 0.1]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(0.583797424337217)

In [34]:
gaussian_probability(0.9, 0.7, 0.1)

np.float64(0.53990966513188)

In [35]:
np.expm1(ann(torch.tensor([[0.643, 0.7, 0.1]]).to(device)).item())

np.float64(0.045898879267076934)

In [36]:
gaussian_probability(0.643, 0.7, 0.1)

np.float64(3.3912431320419234)

In [37]:
np.expm1(ann(torch.tensor([[4, 0.3, 0.2]]).to(device)).item()) # Not really generalized to computation of f(x; mu, sigma), but learned kind of in the ranges the data existed in

np.float64(-0.006184655838678181)

In [38]:
gaussian_probability(4, 0.3, 0.2)

np.float64(9.57716245835995e-75)

In [ ]:
train_dataset.mean.unique()

array([ 0.22517786, -0.39839161, -0.88339008,  0.74481909,  1.40303131,
       -0.84874733, -1.40303131,  0.08660687,  0.25982061,  0.77946184,
        0.6062481 , -0.57160535,  0.15589237,  1.22981757, -1.68017329,
        1.19517482, -1.64553055, -1.43767406,  0.67553359,  1.36838856,
        0.4676771 , -0.74481909, -1.22981757, -1.6108878 , -0.91803283,
        1.29910306, -0.29446336, -0.67553359, -0.64089084, -0.77946184,
        0.64089084, -0.36374886,  0.32910611, -0.15589237, -1.05660382,
        0.91803283,  1.50695955, -1.50695955,  0.81410459, -1.4723168 ,
       -0.22517786, -1.16053207, -0.05196412, -0.19053512,  1.26446031,
       -1.09124657, -0.01732137, -1.36838856,  1.12588932,  1.64553055,
       -1.26446031,  1.71481604, -1.02196108,  0.50231985,  0.84874733,
        0.29446336,  0.43303435,  0.01732137, -0.50231985,  0.05196412,
        1.5416023 ,  0.95267558,  1.16053207, -1.33374581, -0.5369626 ,
        0.57160535,  0.88339008,  1.33374581,  0.71017634,  0.12

In [ ]:
train_dataset.std.unique()

array([ 0.64089084, -0.91803283, -1.19517482,  1.4723168 ,  0.25982061,
        1.64553055, -1.71481604,  1.50695955, -1.57624505, -1.6108878 ,
        1.68017329,  0.15589237, -1.40303131, -0.25982061,  0.22517786,
       -0.5369626 , -1.22981757, -0.81410459, -1.68017329,  0.19053512,
        0.74481909,  1.09124657,  0.32910611, -0.15589237, -0.08660687,
       -1.33374581,  0.36374886,  0.39839161, -0.22517786,  0.67553359,
        0.88339008, -0.12124962,  1.12588932, -0.6062481 , -1.16053207,
        1.43767406, -1.09124657, -0.71017634,  0.4676771 ,  0.91803283,
        1.6108878 ,  0.29446336,  0.84874733,  1.26446031, -0.4676771 ,
        1.40303131,  1.33374581, -1.29910306, -0.77946184,  0.81410459,
        0.43303435,  0.5369626 , -0.01732137,  1.29910306, -1.36838856,
        0.95267558,  1.36838856, -1.12588932, -1.02196108, -0.05196412,
       -1.43767406,  1.5416023 , -0.88339008,  0.01732137, -0.29446336,
        0.57160535, -0.64089084, -0.36374886, -0.84874733, -0.43

In [ ]:
# std 1.25 and mean -3.04
np.expm1(ann(torch.tensor([[-2, -3.04, 1.25]]).to(device)).item())

np.float64(0.0034302099114267215)

In [ ]:
gaussian_probability(-2, -3.04, 1.25)

np.float64(0.22578002723581)

In [ ]:
-3.04 - 3 * 1.25

-6.79

In [ ]:
np.expm1(ann(torch.tensor([[-3.04, -3.04, 0.001]]).to(device)).item())

np.float64(0.009649770411087758)

In [ ]:
gaussian_probability(-3.04, -3.04, 0.001)

np.float64(398.94228040143275)